In [1]:
SEED = 12
BATCH_SIZE = 64
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

In [2]:
print(pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")['Class'].value_counts(normalize=True))

Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

Using cuda device


In [4]:
class CCFDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data.iloc[:, -1]
        self.features = self.data.iloc[:, :-1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32).to()
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return features, label

# Example usage:
dataset = CCFDataset("/kaggle/input/creditcardfraud/creditcard.csv")
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=SEED)



train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,)
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,)
for X, y in test_dl:
    print(f"Shape of X [Time, V1-28, Amount]: {X.shape}")
    print(f"Shape of y [Class]: {y.shape} {y.dtype}")
    break

Shape of X [Time, V1-28, Amount]: torch.Size([64, 30])
Shape of y [Class]: torch.Size([64]) torch.int64


In [5]:
class ModelNN (nn.Module):
    def __init__(self):
        super().__init__()
        self.dense_relu_stack = nn.Sequential(
            nn.Linear(30,64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Linear(64,2),
        )
    def forward(self, x):
        return self.dense_relu_stack(x)
model = ModelNN().to(device)
print(model)        

ModelNN(
  (dense_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 78.039818  [   64/227845]
loss: 0.007973  [ 6464/227845]
loss: 0.085488  [12864/227845]
loss: 0.005275  [19264/227845]
loss: 0.004654  [25664/227845]
loss: 0.003990  [32064/227845]
loss: 0.003488  [38464/227845]
loss: 0.092366  [44864/227845]
loss: 0.093196  [51264/227845]
loss: 0.002830  [57664/227845]
loss: 0.002682  [64064/227845]
loss: 0.002577  [70464/227845]
loss: 0.002516  [76864/227845]
loss: 0.002485  [83264/227845]
loss: 0.002389  [89664/227845]
loss: 0.002292  [96064/227845]
loss: 0.002251  [102464/227845]
loss: 0.002190  [108864/227845]
loss: 0.097498  [115264/227845]
loss: 0.002151  [121664/227845]
loss: 0.002125  [128064/227845]
loss: 0.098686  [134464/227845]
loss: 0.002071  [140864/227845]
loss: 0.002054  [147264/227845]
loss: 0.002008  [153664/227845]
loss: 0.002064  [160064/227845]
loss: 0.099036  [166464/227845]
loss: 0.001929  [172864/227845]
loss: 0.001993  [179264/227845]
loss: 0.001998  [185664/227845]
loss: 0.001935 

In [10]:
##TODO
## Oversampling/Undersampling
## Optimization
## Explore other techniques